In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [6]:
CLIENT_ID = '3C0K5WFDPWWFFQE0V4DIFJ25ZASRONEYEVQUFXMRGX5PIMZ2' # your Foursquare ID
CLIENT_SECRET = '3BIMQSCZXVYZYAKZGBEIKTN3SY0T0N1JRCEIVCC52HUUKA5H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: 3C0K5WFDPWWFFQE0V4DIFJ25ZASRONEYEVQUFXMRGX5PIMZ2


In [7]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d10f941735") # Indian Restaurant category in Foursquare -- reference = https://developer.foursquare.com/docs/build-with-foursquare/categories/
    results[city] = requests.get(url).json()

In [8]:
df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

The Foursquare API Only gives us the nearest 100 venues in the city.
Let's print the maps of the places to get a feel of the density

In [10]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Indian restaurants in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Indian restaurants in New York, NY =  254
Showing Top 100
Total number of Indian restaurants in Chicago, IL =  168
Showing Top 100
Total number of Indian restaurants in San Francisco, CA =  156
Showing Top 100
Total number of Indian restaurants in Jersey City, NJ =  101
Showing Top 100
Total number of Indian restaurants in Boston, MA =  74
Showing Top 100


In [17]:
maps[cities[0]]

In [12]:
maps[cities[1]]

In [13]:
maps[cities[2]]

In [14]:
maps[cities[3]]

In [15]:
maps[cities[4]]

We can see from the data that New York has the most number of Indian resturants followed by Chicago. In the top 100 maps that we have printed, it seems just by looking that New Jersey seems to be most dense with a lot of choices available in close vicinity when it comes to Indian restaurants. Let's confirm this with some concrete statistics.
I will take averages and calculate the distance of the points from the mean to figure out in which city can one roam around with a lot more choices of Indian restaurants than the others.

In [16]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02477456672405344
Chicago, IL
Mean Distance from Mean coordinates
0.05501819287682559
San Francisco, CA
Mean Distance from Mean coordinates
0.02638044252805198
Jersey City, NJ
Mean Distance from Mean coordinates
0.013149012135502226
Boston, MA
Mean Distance from Mean coordinates
0.14840369347692955


As we can see, the mean distance from the mean coordinate is the minimum for Jersey city, so it will be worthwhile for someone who is having a 1-2 day trip to Jersey City to roam around in the city and try out different Indian restuarants. Also, one more interesting point in the data is the number of Indian restaurants in New York City, which is quite high as compared to the others. So, if a person is coming over for a vacation for a week or so in NY city, that would be a really nice opportunity for them to try out different Indian restaurants considering the large number of restuarants available in the city.

This report gives a very basic idea of which places to visit if you are hunting for Indian restaurants based on the density and number of such restaurants available in 5 cities in the US. 
*Happy Indian food hunting* :D